In [5]:
#LIBRARIES NEEDED
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import glm
from statsmodels.genmod.families import Gamma as SM_Gamma, Gaussian as SM_Gaussian, Binomial
from statsmodels.genmod.families.links import log as log_link, identity as id_link, logit
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from pygam import LinearGAM, GammaGAM, LogisticGAM, s, te, l
import geopandas as gpd
from shapely.geometry import Point
from scipy.interpolate import griddata
from pygam import GAM
from pygam import LogisticGAM, s
from functools import reduce
import operator

In [13]:
# Load dataset
file_path = "PHILIPPINES_DATA_19.csv"
df = pd.read_csv(file_path)

# Select and rename key variables
df = df.rename(columns={
    'CR1': 'Age', 'CR2': 'Sex', 'PHR5': 'WeeklyAllowance', 'PHR3': 'Grade',
    'PHR4': 'SchoolType', 'CR5': 'EverSmoked', 'CR7': 'SmokedPast30Days',
    'PHR20': 'TriedVape', 'SR1': 'TriedShisha', 'PHR64': 'ThinksHarmful'
})

# Encode binary responses (Yes/No → 1/0)
binary_vars = ['EverSmoked', 'SmokedPast30Days', 'TriedVape', 'TriedShisha', 'ThinksHarmful']
for var in binary_vars:
    df[var] = df[var].replace({'Yes': 1, 'No': 0}).astype('float')

# Drop rows with missing values in relevant columns
df = df[['SmokedPast30Days', 'Age', 'Sex', 'WeeklyAllowance', 'EverSmoked', 'TriedVape', 'TriedShisha', 'ThinksHarmful']].dropna()
df['Sex'] = df['Sex'].map({1: 'Male', 2: 'Female'})
print("Data ready for analysis:", df.shape)
df.dropna()
df.describe().T

Data ready for analysis: (9249, 8)


,count,mean,std,min,25%,50%,75%,max
SmokedPast30Days,9249.0,1.181749,0.757994,1.0,1.0,1.0,1.0,7.0
Age,9249.0,3.907990,1.492292,1.0,3.0,4.0,5.0,7.0
WeeklyAllowance,9249.0,3.146178,1.100112,1.0,3.0,3.0,4.0,7.0
EverSmoked,9249.0,1.810142,0.392210,1.0,2.0,2.0,2.0,2.0
TriedVape,9249.0,1.794464,0.404114,1.0,2.0,2.0,2.0,2.0
TriedShisha,9249.0,1.974484,0.157696,1.0,2.0,2.0,2.0,2.0
ThinksHarmful,9249.0,1.164666,0.370899,1.0,1.0,1.0,1.0,2.0


In [14]:
df['Sex']

1        Female
2        Female
3        Female
4        Female
5        Female
          ...  
10597      Male
10598      Male
10599      Male
10600      Male
10601      Male
Name: Sex, Length: 9249, dtype: object